In [ ]:
import sys
import os
import glob
import pandas as pd

In [ ]:
file_cluster = glob.glob('../data/clean/cp_hourly/clusters/*.csv')

In [ ]:
generation_existing = pd.read_csv('../data/default/generation_projects_info.tab', sep='\t', 
                         index_col=0, usecols=[0,1,2])
generation_existing.head()

In [ ]:
generation_proposed = pd.read_csv('../generation_test.tab', sep='\t', 
                         index_col=0, usecols=[0,1,2])
generation_proposed.head()

In [ ]:
generation = pd.concat([generation_existing, generation_proposed])
assert len(generation) == (len(generation_proposed) + len(generation_existing))

In [ ]:
generation_ren = generation.loc[generation['gen_tech'].isin(['wind', 'solarpv', 'solarpv_distributed'])]
print (len(generation_ren))

In [ ]:
rens = []
for index, row in generation_ren.iterrows():
    lz = row['gen_load_zone'].split('-')[0]
    gen = row['gen_tech'].split('_')[0]
    proj = 'proj1'
    lz = "{:d}".format(int(lz))
    file = [s for s in file_cluster if lz in s and gen in s and proj in s][0]
    tmp = pd.read_csv(file, index_col=0, parse_dates=True)
    tmp.loc[:, 'project_name'] = index
    if gen == 'solarpv' or gen == 'solarpv_distributed':
        tmp.loc[:, 'capacity_factor'] = tmp.loc[:, 'cf']
    else:
        tmp.loc[:, 'capacity_factor'] = tmp.loc[:,'generation']/2000
    tmp.loc[:, 'time'] = tmp.index.strftime('%m-%d %H:%M:%S')
    tmp.loc[tmp.index.minute == 30, 'time'] = (tmp.index - pd.Timedelta(minutes=30) ).strftime('%m-%d %H:%M:%S')
    tmp.index.names = ['datetime']
    rens.append(tmp)

rens = pd.concat(rens)

In [ ]:
assert len(generation_ren) == len(set(rens['project_name']))

In [ ]:
file_path = os.path.join('../', 'data/clean/SWITCH/')
filename = 'ren-all5.csv'
rens.to_csv(os.path.join(file_path, filename))